In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! git clone https://github.com/fra31/auto-attack.git

Cloning into 'auto-attack'...
remote: Enumerating objects: 672, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 672 (delta 162), reused 159 (delta 141), pack-reused 463
Receiving objects: 100% (672/672), 40.63 MiB | 17.87 MiB/s, done.
Resolving deltas: 100% (413/413), done.


In [3]:
!pip install git+https://github.com/fra31/auto-attack

  Cloning https://github.com/fra31/auto-attack to /tmp/pip-req-build-v_uuu2yc
  Running command git clone --filter=blob:none --quiet https://github.com/fra31/auto-attack /tmp/pip-req-build-v_uuu2yc
  Resolved https://github.com/fra31/auto-attack to commit a39220048b3c9f2cca9a4d3a54604793c68eca7e
  Preparing metadata (setup.py) ... done
  Created wheel for autoattack: filename=autoattack-0.1-py3-none-any.whl size=36229 sha256=e57a47ffd4132a901ea14edd165e9b3f1d57e57a107346a3320f627644ab0193
  Stored in directory: /tmp/pip-ephem-wheel-cache-s6ves0_o/wheels/b7/17/95/4b16fafe1b84fdabd247eb5f01b41165bd98d4c82d64ab93d2
Successfully built autoattack


In [5]:
# Copy Files
!cp /content/drive/MyDrive/ML-Project/Test0.rar /content

In [6]:
! unrar x /content/Test0.rar


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from /content/Test0.rar

Extracting  ckpt.pth                                                       5% 10% 15% 21% 26% 31% 37% 42% 47% 53% 58% 63% 69% 74% 79% 85% 90% 95% 99%  OK 
Extracting  resnet.py                                                     99%  OK 
All OK


In [7]:
!cp -r /content/resnet.py /content/auto-attack/autoattack/examples

In [8]:
%%shell
cat > /content/auto-attack/autoattack/examples/eval.py << EOF

import os
import argparse
from pathlib import Path
import warnings

import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms

import sys
sys.path.insert(0,'..')
from resnet import ResNet34

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, default='./data')
    parser.add_argument('--norm', type=str, default='L2')
    parser.add_argument('--epsilon', type=float, default=0.3)
    #///////////////////////////////////////////////////////////////
    parser.add_argument('--model', type=str, default='/content/ckpt.pth')
    #///////////////////////////////////////////////////////////////
    parser.add_argument('--n_ex', type=int, default=1000)
    parser.add_argument('--individual', action='store_true')
    parser.add_argument('--save_dir', type=str, default='./results')
    parser.add_argument('--batch_size', type=int, default=500)
    parser.add_argument('--log_path', type=str, default='./log_file.txt')
    parser.add_argument('--version', type=str, default='standard')
    parser.add_argument('--state-path', type=Path, default=None)

    args = parser.parse_args()

    # load model
    model = ResNet34()
    #//////////////////////////////////////////////
    model = nn.DataParallel(model)
    loaded = torch.load(args.model)
    model.load_state_dict(loaded['net'])
    #//////////////////////////////////////////////
    model.cuda()
    model.eval()

    # load data
    transform_list = [transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
    transform_chain = transforms.Compose(transform_list)
    item = datasets.CIFAR10(root=args.data_dir, train=False, transform=transform_chain, download=True)
    test_loader = data.DataLoader(item, batch_size=1000, shuffle=False, num_workers=0)

    # create save dir
    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)

    # load attack
    from autoattack import AutoAttack
    adversary = AutoAttack(model, norm=args.norm, eps=args.epsilon, log_path=args.log_path,
        version=args.version)

    l = [x for (x, y) in test_loader]
    x_test = torch.cat(l, 0)
    l = [y for (x, y) in test_loader]
    y_test = torch.cat(l, 0)

    # example of custom version
    if args.version == 'custom':
        adversary.attacks_to_run = ['apgd-ce', 'fab']
        adversary.apgd.n_restarts = 2
        adversary.fab.n_restarts = 2

    # run attack and save images
    with torch.no_grad():
        if not args.individual:
            adv_complete = adversary.run_standard_evaluation(x_test[:args.n_ex], y_test[:args.n_ex],
                bs=args.batch_size, state_path=args.state_path)

            torch.save({'adv_complete': adv_complete}, '{}/{}_{}_1_{}_eps_{:.5f}.pth'.format(
                args.save_dir, 'aa', args.version, adv_complete.shape[0], args.epsilon))

        else:
            # individual version, each attack is run on all test points
            adv_complete = adversary.run_standard_evaluation_individual(x_test[:args.n_ex],
                y_test[:args.n_ex], bs=args.batch_size)

            torch.save(adv_complete, '{}/{}_{}_individual_1_{}_eps_{:.5f}_plus_{}_cheap_{}.pth'.format(
                args.save_dir, 'aa', args.version, args.n_ex, args.epsilon))

/bin/bash: line 86: warning: here-document at line 0 delimited by end-of-file (wanted `EOF')


#AA-L2

##eps = 0.3

In [9]:
epsilon = 0.3
norm = 'L2'
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm
print("Finish!")

100% 170498071/170498071 [00:01<00:00, 90187849.27it/s] 
Extracting ./data/cifar-10-python.tar.gz to ./data
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 95.50%
apgd-ce - 1/2 - 212 out of 500 successfully perturbed
apgd-ce - 2/2 - 190 out of 455 successfully perturbed
robust accuracy after APGD-CE: 55.30% (total time 89.8 s)
apgd-t - 1/2 - 6 out of 500 successfully perturbed
apgd-t - 2/2 - 0 out of 53 successfully perturbed
robust accuracy after APGD-T: 54.70% (total time 569.8 s)
fab-t - 1/2 - 0 out of 500 successfully perturbed
fab-t - 2/2 - 0 out of 47 successfully perturbed
robust accuracy after FAB-T: 54.70% (total time 1485.9 s)
square - 1/2 - 4 out of 500 successfully perturbed
square - 2/2 - 0 out of 47 successfully perturbed
robust accuracy after SQUARE: 54.30% (total time 2791.5 s)
max L2 perturbation: 110.26619, nan in tensor: 0, max: 2.75373, min: -2.42907
robust accuracy: 54.30%
Finish!


##eps = 0.4

In [10]:
epsilon = 0.4
norm = 'L2'
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm
print("Finish!")

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 95.50%
apgd-ce - 1/2 - 213 out of 500 successfully perturbed
apgd-ce - 2/2 - 191 out of 455 successfully perturbed
robust accuracy after APGD-CE: 55.10% (total time 91.5 s)
apgd-t - 1/2 - 5 out of 500 successfully perturbed
apgd-t - 2/2 - 0 out of 51 successfully perturbed
robust accuracy after APGD-T: 54.60% (total time 574.3 s)
fab-t - 1/2 - 0 out of 500 successfully perturbed
fab-t - 2/2 - 0 out of 46 successfully perturbed
robust accuracy after FAB-T: 54.60% (total time 1489.3 s)
square - 1/2 - 4 out of 500 successfully perturbed
square - 2/2 - 0 out of 46 successfully perturbed
robust accuracy after SQUARE: 54.20% (total time 2811.2 s)
max L2 perturbation: 110.26619, nan in tensor: 0, max: 2.75373, min: -2.42907
robust accuracy: 54.20%
Finish!


##eps = 0.5

In [11]:
epsilon = 0.5
norm = 'L2'
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm
print("Finish!")

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 95.50%
apgd-ce - 1/2 - 213 out of 500 successfully perturbed
apgd-ce - 2/2 - 190 out of 455 successfully perturbed
robust accuracy after APGD-CE: 55.20% (total time 91.7 s)
apgd-t - 1/2 - 9 out of 500 successfully perturbed
apgd-t - 2/2 - 0 out of 52 successfully perturbed
robust accuracy after APGD-T: 54.30% (total time 571.4 s)
fab-t - 1/2 - 0 out of 500 successfully perturbed
fab-t - 2/2 - 0 out of 43 successfully perturbed
robust accuracy after FAB-T: 54.30% (total time 1486.7 s)
square - 1/2 - 4 out of 500 successfully perturbed
square - 2/2 - 0 out of 43 successfully perturbed
robust accuracy after SQUARE: 53.90% (total time 2789.3 s)
max L2 perturbation: 110.26619, nan in tensor: 0, max: 2.75373, min: -2.42907
robust accuracy: 53.90%
Finish!


#AA-Linf

##eps = 4/255

In [12]:
epsilon = 4/255
norm = 'Linf'
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm
print("Finish!")

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 95.50%
apgd-ce - 1/2 - 352 out of 500 successfully perturbed
apgd-ce - 2/2 - 314 out of 455 successfully perturbed
robust accuracy after APGD-CE: 28.90% (total time 92.2 s)
apgd-t - 1/1 - 3 out of 289 successfully perturbed
robust accuracy after APGD-T: 28.60% (total time 344.4 s)
fab-t - 1/1 - 0 out of 286 successfully perturbed
robust accuracy after FAB-T: 28.60% (total time 813.6 s)
square - 1/1 - 0 out of 286 successfully perturbed
robust accuracy after SQUARE: 28.60% (total time 1455.7 s)
max Linf perturbation: 2.42907, nan in tensor: 0, max: 2.75373, min: -2.42907
robust accuracy: 28.60%
Finish!


##eps = 6/255

In [13]:
epsilon = 6/255
norm = 'Linf'
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm
print("Finish!")

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 95.50%
apgd-ce - 1/2 - 397 out of 500 successfully perturbed
apgd-ce - 2/2 - 360 out of 455 successfully perturbed
robust accuracy after APGD-CE: 19.80% (total time 92.2 s)
apgd-t - 1/1 - 4 out of 198 successfully perturbed
robust accuracy after APGD-T: 19.40% (total time 269.9 s)
fab-t - 1/1 - 0 out of 194 successfully perturbed
robust accuracy after FAB-T: 19.40% (total time 600.7 s)
square - 1/1 - 0 out of 194 successfully perturbed
robust accuracy after SQUARE: 19.40% (total time 1052.5 s)
max Linf perturbation: 2.42907, nan in tensor: 0, max: 2.75373, min: -2.42907
robust accuracy: 19.40%
Finish!


##eps = 8/255

In [14]:
epsilon = 8/255
norm = 'Linf'
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm
print("Finish!")

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 95.50%
apgd-ce - 1/2 - 430 out of 500 successfully perturbed
apgd-ce - 2/2 - 397 out of 455 successfully perturbed
robust accuracy after APGD-CE: 12.80% (total time 92.2 s)
apgd-t - 1/1 - 3 out of 128 successfully perturbed
robust accuracy after APGD-T: 12.50% (total time 205.8 s)
fab-t - 1/1 - 0 out of 125 successfully perturbed
robust accuracy after FAB-T: 12.50% (total time 419.6 s)
square - 1/1 - 0 out of 125 successfully perturbed
robust accuracy after SQUARE: 12.50% (total time 711.0 s)
max Linf perturbation: 2.42907, nan in tensor: 0, max: 2.75373, min: -2.42907
robust accuracy: 12.50%
Finish!
